In [119]:
import pandas as pd
import numpy as np
from matplotlib import pyplot
from sklearn.impute import SimpleImputer

import warnings
warnings.filterwarnings('ignore')

# Исследование данных

In [134]:
# load the data
data_train = pd.read_csv('data/train.csv')
data_test = pd.read_csv('data/test.csv')
submission = pd.read_csv('data/submission.csv')

# Check random row
random_number_train = np.random.randint(data_train.shape[0])
random_number_test = np.random.randint(data_test.shape[0])
#print(data_train.iloc[random_number_train])
#print(data_test.iloc[random_number_test])

# Select target
y_train = data_train.Churn
y_test = submission

# Select features
X_train = data_train.drop('Churn', axis=1)
X_test = data_test

#Check missing values
X_train.info()
X_test.info()

cols_with_missing_X_train = [col for col in X_train
                     if X_train[col].isnull().any() or (X_train[col] == ' ').any()]
cols_with_missing_X_test = [col for col in X_test
                     if X_test[col].isnull().any() or (X_test[col] == ' ').any()]
print('Cols with missing values in X_train:', cols_with_missing_X_train)
print('Cols with missing values in X_test:', cols_with_missing_X_test)

# Imputation
for i, val in enumerate(X_train.TotalSpent):
    if val == ' ':
        X_train.TotalSpent.iloc[i] = np.nan
X_train.astype({"TotalSpent": "float64"})
my_imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5282 entries, 0 to 5281
Data columns (total 19 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   ClientPeriod              5282 non-null   int64  
 1   MonthlySpending           5282 non-null   float64
 2   TotalSpent                5282 non-null   object 
 3   Sex                       5282 non-null   object 
 4   IsSeniorCitizen           5282 non-null   int64  
 5   HasPartner                5282 non-null   object 
 6   HasChild                  5282 non-null   object 
 7   HasPhoneService           5282 non-null   object 
 8   HasMultiplePhoneNumbers   5282 non-null   object 
 9   HasInternetService        5282 non-null   object 
 10  HasOnlineSecurityService  5282 non-null   object 
 11  HasOnlineBackup           5282 non-null   object 
 12  HasDeviceProtection       5282 non-null   object 
 13  HasTechSupportAccess      5282 non-null   object 
 14  HasOnlin

ValueError: Cannot use mean strategy with non-numeric data:
could not convert string to float: 'Male'